In [1]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import lxml
import re

### Scraping the data and building the data frame

Procedure adapted from https://towardsdatascience.com/web-scraping-scraping-table-data-1665b6b2271c.

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
text_data = rq.get(url).text
soup = BeautifulSoup(text_data, "lxml")
#print(soup.prettify())

In [3]:
data=soup.find_all("table", attrs={"class":"wikitable"})
print("{} tables found".format(len(data)))


1 tables found


In [4]:
table=data[0]
body = table.find_all("tr")
head = body[0]
body_rows = body[1:]

headings = []
for item in head.find_all("th"):
    item = (item.text).rstrip("\n")
    headings.append(item)
print(headings)

['Postal Code', 'Borough', 'Neighbourhood']


In [5]:
body_rows[0].text

'\nM1A\n\nNot assigned\n\nNot assigned\n'

In [6]:
all_rows = []

for row_num in range (len(body_rows)):
    row = []
    for row_item in body_rows[row_num].find_all("td"):
        datum = re.sub("(\xa0)|(\n),","",row_item.text)
        datum = re.sub("\n","",datum)
        row.append(datum)
    all_rows.append(row)

In [7]:
all_rows[0:5]

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront']]

In [8]:
df = pd.DataFrame(data=all_rows,columns=headings)
print(df.head())
print("size of data frame: ",df.shape)

  Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront
size of data frame:  (180, 3)


We identify those Postal Codes which have no borough assigned. These entries will be removed.

In [9]:
df[df["Borough"]=="Not assigned"]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
7,M8A,Not assigned,Not assigned
10,M2B,Not assigned,Not assigned
15,M7B,Not assigned,Not assigned
...,...,...,...
174,M4Z,Not assigned,Not assigned
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned


In [10]:
df_assigned =df[df["Borough"]!="Not assigned"]
df_assigned.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Now only postal codes with assigned boroughs remain. To be safe, are there any remaining Neighbourhood entries reading "Not assigned"?

In [11]:
df=df_assigned
df[df["Neighbourhood"]=="Not assigned"]

,Postal Code,Borough,Neighbourhood


Finally, let's look at the shape of our data frame of postal codes and their related neighbourhoods and boroughs.

In [12]:
df.reset_index()
print("Shape of final data frame: ", df.shape)

Shape of final data frame:  (103, 3)


## Acquiring coordinates

Now we will add to the data frame the coordinates associated with each borough/postal code.  
We have two ways of doing this: 
1) use ghe google geocoder tool to read in location names and output coordinates   
2) use an existing dataframe of postal codes with corresponding coordinates

The first approach did not work for me. The geocoder did not return any coordinates after running for quite some time. I include the code here in hope that it will be useful later, or that someone can help identify the problem.  
We proceed with the second approach, reading in the geospatial data and performing a merge (inner join) with the processed data frame created above.

<!-- import sys
!{sys.executable} -m pip install geocoder
import geocoder -->

<!-- indices=df.index
lats=[]
longs=[]

for index in indices:
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(df.iloc[index]["Postal Code"]))
        lat_lng_coords = g.latlng
    lats.append(lat_lng_coords[0])
    longs.append(lat_lng_coords[1]) -->

Read in data from the web...

In [13]:
latlng_df = pd.read_csv('https://cocl.us/Geospatial_data', index_col='Postal Code')

In [25]:
#examine what we've imported...
latlng_df.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [26]:
#...and check that the shape makes sense.
print(latlng_df.shape)

(103, 2)


In [17]:
#merge performs inner join, using the shared column "Postal Code"
total_df=df.merge(latlng_df, on = "Postal Code")
total_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [22]:
print("The final data frame with coordinates, boroughs and neighbourhoods has shape ",total_df.shape, "as expected.")

The final data frame with coordinates, boroughs and neighbourhoods has shape  (103, 5) as expected.


## Clustering Postal Codes

I'm going to follow the general procedure of the New York clustering from a previous exercise, but using postal codes in the North York district of Toronto. I will cluster postal codes according to their venue types.  
  
I'll start by importing required libraries and creating a new data frame containing only the North York venues.

In [98]:
import requests # library to handle requests
#import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


#!pip install geopy  #UNCOMMENT if geopy not installed!
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

from sklearn.cluster import KMeans

#! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [119]:
northyork_df=total_df[total_df["Borough"]=="North York"]
#nyork_df.set_index("Postal Code",inplace=True) #I thought this was a good idea. It was not.
northyork_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
7,M3B,North York,Don Mills,43.745906,-79.352188
10,M6B,North York,Glencairn,43.709577,-79.445073


In [120]:
# The code was removed by Watson Studio for sharing.

Borrow function for looping over all postal codes from New York clustering exercise...

In [88]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Code', 
                  'Code Latitude', 
                  'Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [89]:
northyork_venues = getNearbyVenues(names=northyork_df["Postal Code"], 
                                   latitudes=northyork_df["Latitude"],
                                  longitudes=northyork_df["Longitude"],
                                  )

In [90]:
print(northyork_venues.shape)
northyork_venues.head()

(202, 7)


,Code,Code Latitude,Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [104]:
northyork_venues.groupby('Code').count().head()

,Code Latitude,Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Code,,,,,,
M2H,6,6,6,6,6,6
M2J,30,30,30,30,30,30
M2K,4,4,4,4,4,4
M2M,1,1,1,1,1,1
M2N,30,30,30,30,30,30
M2P,4,4,4,4,4,4
M2R,5,5,5,5,5,5
M3A,2,2,2,2,2,2
M3B,4,4,4,4,4,4


How many unique venue categories are there?

In [94]:
len(northyork_venues['Venue Category'].unique())

91

North York seems to have far fewer types of venues (91) than Manhatten does (329).  

I'm going to one-hot encode the venue types for each neighbourhood.

In [95]:
northyork_onehot = pd.get_dummies(northyork_venues[["Venue Category"]], prefix="", prefix_sep="")

northyork_onehot['Postal Code'] = northyork_venues['Code']
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]

northyork_onehot.head()

,Postal Code,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
northyork_grouped = northyork_onehot.groupby("Postal Code").mean().reset_index()
northyork_grouped.head()

,Postal Code,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant
0,M2H,0.0,0.0,0.000000,0.0,0.000000,0.0,0.166667,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,M2J,0.0,0.0,0.033333,0.0,0.000000,0.0,0.000000,0.033333,0.066667,...,0.0,0.000000,0.0,0.000000,0.0,0.033333,0.033333,0.0,0.033333,0.000000
2,M2K,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.250000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
3,M2M,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
4,M2N,0.0,0.0,0.000000,0.0,0.033333,0.0,0.000000,0.000000,0.033333,...,0.0,0.033333,0.0,0.033333,0.0,0.000000,0.000000,0.0,0.000000,0.033333


Double-click and run this cell to get top n venues in each Postal Code

<!--
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n') -->

Now I'll cluster the neighbourhoords by venue type frequency. 

In [101]:
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(len(kmeans.labels_))
kmeans.labels_

23


array([0, 0, 4, 2, 0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       3], dtype=int32)

In [117]:
#pd.DataFrame(data=kmeans.labels_,index = [0:23],columns = "K-Means label")
clustered_df = northyork_grouped[["Postal Code"]]
clustered_df["K-Means labels"]=kmeans.labels_.tolist()
clustered_df

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,Postal Code,K-Means labels
0,M2H,0
1,M2J,0
2,M2K,4
3,M2M,2
4,M2N,0
5,M2P,0
6,M2R,0
7,M3A,2
8,M3B,4
9,M3C,0
